In [ ]:
!./build.sh

In [ ]:
!./format.sh
!echo "=== Testing Removing Unused Import and Varialbe ==="
!autoflake --in-place --remove-unused-variables --remove-all-unused-imports health_check.py

In [ ]:
import os

from deploykit import (
    build_package,
    upload_package, 
    deploy_pacakge, 
    update_package_code,
    update_env_vars
)

# Test Lambda

In [ ]:
from health_check import handler

In [ ]:
test_event = {}
response = handler(event=test_event, context=_)
assert response['statusCode'] == 200 

# Deploy Lambda Handler

In [ ]:
# !pip install --upgrade -r requirements.txt --target ./packages

In [ ]:
BUCKET = os.environ['BUCKET']
BUCKET_KEY = "lambdas/health-check.zip"
FUNCTION_NAME = "quack-tsln-health-check"
ENV_DYNAMODB = os.environ["TABLE"]
ENV_VARIABLES = {'Variables': {'BUCKET': BUCKET, 'DYNAMODB': ENV_DYNAMODB, "ENVIRON": "prod"}}
CODE_INFO = {"S3Bucket": BUCKET, "S3Key": BUCKET_KEY}
PACKAGE = BUCKET_KEY.replace("lambdas/", "")

In [ ]:
build_package(PACKAGE)

In [ ]:
upload_package(PACKAGE, BUCKET, BUCKET_KEY)

In [ ]:
DESC = "Health check lambda"
MAIN_PYFILE = "health_check"

In [ ]:
try:
    deploy_pacakge(
        function_name=FUNCTION_NAME, 
        handler=MAIN_PYFILE,
        timeout=180, 
        desc=DESC, 
        env_vars=ENV_VARIABLES, 
        code_info=CODE_INFO,
        memory_size=128
    )
except Exception as e:
    print(e)
    update_package_code(function_name=FUNCTION_NAME, bucket=BUCKET, key=BUCKET_KEY)

In [ ]:
update_env_vars(function_name=FUNCTION_NAME, envvars=ENV_VARIABLES)